<a href="https://colab.research.google.com/gist/osaaso3/89b593a6054893900c809721054f075b/pyactrccm2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

pyACTrCCM V3

Hello R...

In [ ]:
pip install python_actr

In [ ]:
import python_actr
from python_actr import Model, ACTR, Buffer, log_everything, log
from python_actr import Memory
from python_actr import DMSpreading
#from python_actr import log_everything

In [ ]:
class MyEnvironment(Model):
    pass
class MyAgent(ACTR):
   
    focus=Buffer()
    focus.set('sandwich_step:bread_bottom')

    def bread_bottom(focus='sandwich_step:bread_bottom'):# if focus buffer has this chunk then....
        print("I have a piece of bread")                 # print the action
        focus.set('sandwich_step:cheese')                # change chunk in focus buffer

    def cheese(focus='sandwich_step:cheese'):            # the rest of the productions are the same
        print("I have put cheese on the bread")          # but carry out different actions
        focus.set('sandwich_step:ham')

    def ham(focus='sandwich_step:ham'):
        print("I have put  ham on the cheese")
        focus.set('sandwich_step:bread_top')

    def bread_top(focus='sandwich_step:bread_top'):
        print("I have put bread on the ham")
        print("I have made a ham and cheese sandwich")
        focus.set('sandwich_step:stop')   

tim=MyAgent()                              # name the agent
subway=MyEnvironment()                     # name the environment
subway.agent=tim                           # put the agent in the environment
log_everything(subway)
subway.run() 

   0.000 agent.production_match_delay 0
   0.000 agent.production_threshold None
   0.000 agent.production_time 0.05
   0.000 agent.production_time_sd None
   0.000 agent.production bread_bottom
   0.050 agent.production None
I have a piece of bread
   0.050 agent.focus.chunk sandwich_step:cheese
   0.050 agent.production cheese
   0.100 agent.production None
I have put cheese on the bread
   0.100 agent.focus.chunk sandwich_step:ham
   0.100 agent.production ham
   0.150 agent.production None
I have put  ham on the cheese
   0.150 agent.focus.chunk sandwich_step:bread_top
   0.150 agent.production bread_top
   0.200 agent.production None
I have put bread on the ham
I have made a ham and cheese sandwich
   0.200 agent.focus.chunk sandwich_step:stop


In [ ]:
#Complex fan retrieval
# Kam Kwok, 2014
##import ccm
##from ccm.lib.actr import *
#log=log(html=True)

class CFanModel(ACTR):
    goal=Buffer()
    retrieval=Buffer()
    img=Buffer()
##    memory=Memory(retrieval,latency=0.63)
    memory=Memory(retrieval,latency=0.05,threshold=-25,maximum_time=20,finst_size=10,finst_time=0.0)
    spread=DMSpreading(memory,goal)
    spread.strength=1
    spread.weight[goal]=0.33
###Turning on spreading action and inhibition
##  memory=Memory(retrieval,latency=0.05,threshold=-25,maximum_time=20,finst_size=10,finst_time=0.0) # this line is copied from simple inhibition example      
##  dm_n=DMNoise(memory,noise=0.0,baseNoise=0.0)    # turn on for DM subsymbolic processing
##  dm_bl=DMBaseLevel(memory,decay=0.5,limit=None)  # turn on for DM subsymbolic processing; decay is default to 0.5 based on experiments
##  spread=DMSpreading(memory,goal)                  # turn on spreading activation for DM from goal buffer
##  spread.strength=1                             # set strength of activation for buffers
##  spread.weight[goal]=.33                       # set weight to adjust for how many slots in the buffer
##                                                 # usually this is strength divided by number of slots
    def init():
        memory.add('ring in bucket')
        memory.add('ring in box')
        memory.add('box in pool')

# probe is ?obj in ?location
    def start_obj(goal='test ?obj ?location'):
        memory.request('?obj in ?') #get contianer in retrieval
        goal.set('getcontainer ?obj ?location')
    
    def getcontainer(goal='getcontainer ?obj ?location',
                   retrieval='?obj in ?container'):
        retrieval.clear()
        memory.request('?container in ?')
        goal.set('respond ?obj ?location')

    def respond_yes(goal='respond ?obj ?location',
                  retrieval='?container in ?location'):
        print ('Yes')
        retrieval.clear()
        goal.clear()

    def respond_retry (goal='respond ?obj ?location',
                      retrieval='?container in !?location'):
        img.set('?container')
        print ("re-try", container)
        memory.request('?container in ?')
        goal.set('respond ?obj ?location')
##        retrieval.clear()
##        goal.set('test ?obj ?location')
  
    def respond_no(goal='respond ?obj ?location',
                      img='?container',
                      retrieval='?container in !?location'):
        print ('No')
        retrieval.clear()
        goal.clear()

    def mem_error(goal='respond ?obj ?location', memory='error:True'):
        print ("Cannot recall any fact...")
        goal.set('test ?obj ?location')


model=CFanModel()
log_everything(model)
##for item in ['test ring pool','test hippie park']:
##    model.goal.set(item)
##    model.run()
##model.goal.set('test ring bank')
##model.run()
model.goal.set('test ring pool')
model.run()
model.goal.set('test ring bank')
model.run()

   0.000 production_match_delay 0
   0.000 production_threshold None
   0.000 production_time 0.05
   0.000 production_time_sd None
   0.000 goal.chunk None
   0.000 img.chunk None
   0.000 memory.error False
   0.000 memory.busy False
   0.000 memory.latency 0.05
   0.000 memory.threshold -25
   0.000 memory.maximum_time 20
   0.000 memory.record_all_chunks False
   0.000 retrieval.chunk None
   0.000 goal.chunk test ring pool
   0.000 production start_obj
   0.050 production None
   0.050 memory.busy True
   0.050 goal.chunk getcontainer ring pool
   0.102 retrieval.chunk ring in bucket
   0.102 memory.busy False
   0.102 production getcontainer
   0.152 production None
   0.152 retrieval.chunk None
   0.152 memory.busy True
   0.152 goal.chunk respond ring pool
  20.152 memory.error True
  20.152 retrieval.chunk None
  20.152 memory.busy False
  20.152 production mem_error
  20.202 production None
Cannot recall any fact...
  20.202 goal.chunk test ring pool
  20.202 production start

In [ ]:
#import sys
#sys.path.append('/Users/robertwest/CCMSuite')
#sys.path.append('C:\Users\rlwes_000\Documents\GitHub\ccmsuite')

#import ccm      
#log=ccm.log()   

#from ccm.lib.actr import *  

class MyEnvironment(Model):
    pass

class MyAgent(ACTR):
    focus=Buffer()
    Imagebuffer=Buffer
    Visionbuffer=Buffer()
    DMbuffer=Buffer()
    DM=Memory(DMbuffer)

    
    def init():
        DM.add('isa:objectcontainer objectx:apple container:bowl')
        DM.add('isa:objectcontainer objectx:apple container:bucket')
        DM.add('isa:objectcontainer objectx:apple container:bin')
        
        DM.add('isa:containerlocation container:bowl location:house')
        DM.add('isa:containerlocation container:bucket location:store')
        DM.add('isa:containerlocation container:bin location:park')
        
        focus.set('status:start')
        Visionbuffer.set('isa:objectlocation objectx:apple location:house')


    #start trial
    def start(focus='status:start', Visionbuffer='isa:objectlocation objectx:?objectx'):
        print ("recalling based on objects")
        DM.request('isa:objectcontainer objectx:?objectx container:?')
        focus.set('status:get_container') 


    #recall a container for the target object
    def container(focus='status:get_container', 
                  DMbuffer='isa:objectcontainer objectx:?objectx container:?container'):  
        #print objectx
        #print "is in ......."         
        print container
        DM.request('isa:containerlocation container:?container location:?')
        focus.set('status:get_location')
        

    #recall a location for the container the object is in
    def location(focus='status:get_location', 
                 DMbuffer='isa:containerlocation container:?container location:?location'):  
        print container
        print "is in ......."         
        print location
        focus.set('status:check_location')


    #success
    def yes_location(focus='status:check_location', 
                     DMbuffer='container:?container location:?location', Visionbuffer='location:?location'):  
        print 'YES'
        focus.set('status:stop')


    #failure
    def no_location(focus='status:check_location', 
                    DMbuffer='container:?container location:?location', Visionbuffer='location:!?location'):  
        print 'NO'
        focus.set('status:stop')



tim=MyAgent()                              # name the agent
subway=MyEnvironment()                     # name the environment
subway.agent=tim                           # put the agent in the environment
ccm.log_everything(subway)                 # print out what happens in the environment

subway.run()                               # run the environment
ccm.finished()                             # stop the environment

SyntaxError: ignored